In [1]:
import torchvision.models as models
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.optim import Adam
from tkr_generator import *
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

class channel1(nn.Module):
	def __init__(self):
		super(channel1, self).__init__()
		self.conv1 = nn.Conv2d(1, 96, kernel_size=3, stride=1, padding=1)
		self.relu1 = nn.ReLU(inplace=True)
		self.conv2 = nn.Conv2d(96, 10, kernel_size=3, stride=1, padding=1)
		self.conv3 = nn.Conv2d(10, 4, kernel_size=3, stride=1, padding=1)
		self.fc1 = nn.Linear(16 * 5 * 5, 120)
		self.pool = nn.MaxPool2d(2, 2)
		self.fc1 = nn.Linear(4096, 120)
		self.fc2 = nn.Linear(120, 1)

	def forward(self, x):
		x = self.conv1(x)
		x = self.pool(self.relu1(x))
		
		x = self.conv2(x)
		x = self.pool(self.relu1(x))

		x = self.conv3(x)
		x = self.pool(self.relu1(x))


		x = torch.flatten(x, 1)
		x = self.fc1(x)
		x = self.fc2(x)

		return x


In [3]:
model = channel1()
model.load_state_dict(torch.load('tmp_weights.pth'))
model.eval()

channel1(
  (conv1): Conv2d(1, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(96, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(10, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=120, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc2): Linear(in_features=120, out_features=1, bias=True)
)

In [4]:
train_loader = TKRDataset('R:\OAIKNEE-Q1126\OAI_UQ_processed\AKOA_TKR_Data', 1)

In [44]:
images, labels = train_loader.__getitem__(0)
images = torch.tensor(images, dtype=float)
outputs = model(images.reshape(4,1,256,256).float())
pred = torch.round(torch.sigmoid(outputs)).detach().numpy().squeeze()

In [46]:
np.sum(pred == labels)

3

In [17]:
torch.sum(labels == pred)

TypeError: sum(): argument 'input' (position 1) must be Tensor, not bool

In [47]:
count = 0
for i in range(100):
    images, labels = train_loader.__getitem__(0)
    images = torch.tensor(images, dtype=float)
    outputs = model(images.reshape(4,1,256,256).float())
    pred = torch.round(torch.sigmoid(outputs)).detach().numpy().squeeze()
    count += np.sum(pred == labels)

In [59]:
# Param = {
#     'LR': 0.0001,
#     'WC': 0.0001
# }
# num_epochs = 1
# model_type = 'densenet'
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# print("The model will be running on", device, "device")
# # Convert model parameters and buffers to CPU or Cuda
# model = channel1().float()
# loss_fn = nn.BCELoss()
# optimizer = Adam(model.parameters(), lr=Param['LR'], weight_decay=Param['WC'])
# best_accuracy = 0.0

# print(os.getcwd())
# train_loader = TKRDataset('R:\OAIKNEE-Q1126\OAI_UQ_processed\AKOA_TKR_Data', 0)


# Define your execution device

for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        # get the inputs
        images = torch.tensor(images, dtype=float).to(device, dtype=float)
        labels = torch.tensor(labels, dtype=float).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(images.reshape(4,1,256,256).float())
        pred = torch.sigmoid(outputs)
        
        # compute the loss based on model output and real labels
        loss = loss_fn(pred[:,0], labels.float())
        # backpropagate the loss
        loss.backward()
        # adjust parameters based on the calculated gradients
        optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        
        if i % 20 == 0 and not i== 0:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
            # zero the loss
            if running_loss / 20 < 0.5:
                los = running_loss /20
                torch.save(model.state_dict(), f'sagittal_weights_iter{i}_loss{los}.pth')
            running_loss = 0.0  

[1,    21] loss: 0.605
[1,    41] loss: 0.610
[1,    61] loss: 0.633
[1,    81] loss: 0.599
[1,   101] loss: 0.663
[1,   121] loss: 0.601
[1,   141] loss: 0.548
[1,   161] loss: 0.660
[1,   181] loss: 0.577
[1,   201] loss: 0.592
[1,   221] loss: 0.598
[1,   241] loss: 0.518
[1,   261] loss: 0.665
[1,   281] loss: 0.637
[1,   301] loss: 0.557
[1,   321] loss: 0.698
[1,   341] loss: 0.656
[1,   361] loss: 0.597
[1,   381] loss: 0.654
[1,   401] loss: 0.605
[1,   421] loss: 0.702
[1,   441] loss: 0.574
[1,   461] loss: 0.543
[1,   481] loss: 0.651
[1,   501] loss: 0.582
[1,   521] loss: 0.629
[1,   541] loss: 0.536
[1,   561] loss: 0.663
[1,   581] loss: 0.595
[1,   601] loss: 0.622
[1,   621] loss: 0.634
[1,   641] loss: 0.620
[1,   661] loss: 0.620
[1,   681] loss: 0.561
[1,   701] loss: 0.562
[1,   721] loss: 0.638
[1,   741] loss: 0.590
[1,   761] loss: 0.552
[1,   781] loss: 0.591
[1,   801] loss: 0.583
[1,   821] loss: 0.534
[1,   841] loss: 0.569
[1,   861] loss: 0.642
[1,   881] 

In [60]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        # get the inputs
        images = torch.tensor(images, dtype=float).to(device, dtype=float)
        labels = torch.tensor(labels, dtype=float).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(images.reshape(4,1,256,256).float())
        pred = torch.sigmoid(outputs)
        
        # compute the loss based on model output and real labels
        loss = loss_fn(pred[:,0], labels.float())
        # backpropagate the loss
        loss.backward()
        # adjust parameters based on the calculated gradients
        optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        
        if i % 20 == 0 and not i== 0:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
            # zero the loss
            if running_loss / 20 < 0.5:
                los = running_loss /20
                torch.save(model.state_dict(), f'sagittal_weights_iter{i}_loss{los}.pth')
            running_loss = 0.0  

[1,    21] loss: 0.564
[1,    41] loss: 0.577
[1,    61] loss: 0.600
[1,    81] loss: 0.673
[1,   101] loss: 0.569
[1,   121] loss: 0.594
[1,   141] loss: 0.613
[1,   161] loss: 0.680


In [61]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        # get the inputs
        images = torch.tensor(images, dtype=float).to(device, dtype=float)
        labels = torch.tensor(labels, dtype=float).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(images.reshape(4,1,256,256).float())
        pred = torch.sigmoid(outputs)
        
        # compute the loss based on model output and real labels
        loss = loss_fn(pred[:,0], labels.float())
        # backpropagate the loss
        loss.backward()
        # adjust parameters based on the calculated gradients
        optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        
        if i % 20 == 0 and not i== 0:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
            # zero the loss
            if running_loss / 20 < 0.5:
                los = running_loss /20
                torch.save(model.state_dict(), f'sagittal_weights_iter{i}_loss{los}.pth')
            running_loss = 0.0  

[1,    21] loss: 0.630
[1,    41] loss: 0.605
[1,    61] loss: 0.520
[1,    81] loss: 0.549
[1,   101] loss: 0.494
[1,   121] loss: 0.548
[1,   141] loss: 0.466
[1,   161] loss: 0.568
[1,   181] loss: 0.645
[1,   201] loss: 0.573
[1,   221] loss: 0.580
[1,   241] loss: 0.579


In [58]:
count = 0
for i in range(100):
    images, labels = train_loader.__getitem__(0)
    images = torch.tensor(images, dtype=float)
    outputs = model(images.reshape(4,1,256,256).float())
    pred = torch.round(torch.sigmoid(outputs)).detach().numpy().squeeze()
    count += np.sum(pred == labels)
count

267

In [62]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        # get the inputs
        images = torch.tensor(images, dtype=float).to(device, dtype=float)
        labels = torch.tensor(labels, dtype=float).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(images.reshape(4,1,256,256).float())
        pred = torch.sigmoid(outputs)
        
        # compute the loss based on model output and real labels
        loss = loss_fn(pred[:,0], labels.float())
        # backpropagate the loss
        loss.backward()
        # adjust parameters based on the calculated gradients
        optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        
        if i % 20 == 0 and not i== 0:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
            # zero the loss
            if running_loss / 20 < 0.5:
                los = running_loss /20
                torch.save(model.state_dict(), f'sagittal_weights_iter{i}_loss{los}.pth')
            running_loss = 0.0  

In [52]:
256/400

0.64

In [63]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        # get the inputs
        images = torch.tensor(images, dtype=float).to(device, dtype=float)
        labels = torch.tensor(labels, dtype=float).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(images.reshape(4,1,256,256).float())
        pred = torch.sigmoid(outputs)
        
        # compute the loss based on model output and real labels
        loss = loss_fn(pred[:,0], labels.float())
        # backpropagate the loss
        loss.backward()
        # adjust parameters based on the calculated gradients
        optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        
        if i % 20 == 0 and not i== 0:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
            # zero the loss
            if running_loss / 20 < 0.5:
                los = running_loss /20
                torch.save(model.state_dict(), f'sagittal_weights_iter{i}_loss{los}.pth')
            running_loss = 0.0  

[1,    21] loss: 0.594
[1,    41] loss: 0.524
[1,    61] loss: 0.599
[1,    81] loss: 0.580
[1,   101] loss: 0.551
[1,   121] loss: 0.576
[1,   141] loss: 0.598
[1,   161] loss: 0.584
[1,   181] loss: 0.518
[1,   201] loss: 0.493
[1,   221] loss: 0.525
[1,   241] loss: 0.683
[1,   261] loss: 0.634
[1,   281] loss: 0.558
[1,   301] loss: 0.644
[1,   321] loss: 0.519
[1,   341] loss: 0.631
[1,   361] loss: 0.584
[1,   381] loss: 0.613
[1,   401] loss: 0.544
[1,   421] loss: 0.544
[1,   441] loss: 0.611
[1,   461] loss: 0.610
[1,   481] loss: 0.566
[1,   501] loss: 0.626
[1,   521] loss: 0.520
[1,   541] loss: 0.583
[1,   561] loss: 0.577
[1,   581] loss: 0.562
[1,   601] loss: 0.526
[1,   621] loss: 0.567
[1,   641] loss: 0.661
[1,   661] loss: 0.615
[1,   681] loss: 0.583
[1,   701] loss: 0.570
[1,   721] loss: 0.601
[1,   741] loss: 0.514
[1,   761] loss: 0.441
[1,   781] loss: 0.546
[1,   801] loss: 0.523
[1,   821] loss: 0.495
[1,   841] loss: 0.521
[1,   861] loss: 0.693
[1,   881] 

In [64]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        # get the inputs
        images = torch.tensor(images, dtype=float).to(device, dtype=float)
        labels = torch.tensor(labels, dtype=float).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(images.reshape(4,1,256,256).float())
        pred = torch.sigmoid(outputs)
        
        # compute the loss based on model output and real labels
        loss = loss_fn(pred[:,0], labels.float())
        # backpropagate the loss
        loss.backward()
        # adjust parameters based on the calculated gradients
        optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        
        if i % 20 == 0 and not i== 0:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
            # zero the loss
            if running_loss / 20 < 0.5:
                los = running_loss /20
                torch.save(model.state_dict(), f'sagittal_weights_iter{i}_loss{los}.pth')
            running_loss = 0.0  

[1,    21] loss: 0.539
[1,    41] loss: 0.564
[1,    61] loss: 0.568
[1,    81] loss: 0.754
[1,   101] loss: 0.645
[1,   121] loss: 0.579
[1,   141] loss: 0.615
[1,   161] loss: 0.500
[1,   181] loss: 0.597
[1,   201] loss: 0.490
[1,   221] loss: 0.474
[1,   241] loss: 0.625
[1,   261] loss: 0.513
[1,   281] loss: 0.509
[1,   301] loss: 0.626
[1,   321] loss: 0.504
[1,   341] loss: 0.478
[1,   361] loss: 0.582
[1,   381] loss: 0.560
[1,   401] loss: 0.531
[1,   421] loss: 0.591
[1,   441] loss: 0.493
[1,   461] loss: 0.570
[1,   481] loss: 0.586
[1,   501] loss: 0.546
[1,   521] loss: 0.569
[1,   541] loss: 0.549
[1,   561] loss: 0.515
[1,   581] loss: 0.609
[1,   601] loss: 0.645
[1,   621] loss: 0.497
[1,   641] loss: 0.466
[1,   661] loss: 0.459
[1,   681] loss: 0.591
[1,   701] loss: 0.587
[1,   721] loss: 0.482
[1,   741] loss: 0.648
[1,   761] loss: 0.554
[1,   781] loss: 0.635
[1,   801] loss: 0.590
[1,   821] loss: 0.454
[1,   841] loss: 0.541
[1,   861] loss: 0.572
[1,   881] 